In [17]:
import pandas as pd
import json
import time
from random import randrange
import requests

In [18]:
base_url = "https://api.fiscaldata.treasury.gov/services/api/fiscal_service/v2/accounting/od/debt_to_penny?filter=record_date:gte:2014-12-10,record_date:lte:2024-12-09&sort=-record_date&page%5Bnumber%5D="
end_url = "&page%5Bsize%5D=100"

In [ ]:
target_link = [f"{base_url}{number}{end_url}" for number in range(1,30)]
target_link

In [ ]:
page_number = 1
valid = True
lister = []
all_df_data = []

while valid == True:
  link = f"{base_url}{page_number}{end_url}"
  page_number +=1
  try:
    response = requests.get(link)
    if response.status_code == 200:
      print(f"scraping {link} link")
      lister.append(link)
      text = response.text
      data = json.loads(text)
      df_all = pd.json_normalize(data)
      df_data = pd.json_normalize(data["data"])
      remove_columns = ['debt_held_public_amt',	'intragov_hold_amt', 'src_line_nbr', 'record_fiscal_quarter', 'record_calendar_quarter', 'record_calendar_month']
      df_data = df_data.drop(columns=remove_columns)
      all_df_data.append(df_data)
    else:
      print(f"link is unscrapable {link}")
      valid = False
  except requests.exceptions.RequestException as e:
    print(e)
  
  snoozer = randrange(1,2)
  time.sleep(snoozer)
  print(f"snoozing for {snoozer} seconds")

print(f"Done Scraping!! {page_number - 1} links")

In [ ]:
all_df_data

In [59]:
output = pd.concat(all_df_data, ignore_index=True)
output


,record_date,tot_pub_debt_out_amt,record_fiscal_year,record_calendar_year,record_calendar_day
0,2024-12-09,36172195279062.94,2025,2024,09
1,2024-12-06,36166037570457.33,2025,2024,06
2,2024-12-05,36167364938108.44,2025,2024,05
3,2024-12-04,36170511547438.85,2025,2024,04
4,2024-12-03,36169413863866.88,2025,2024,03
...,...,...,...,...,...
2506,2014-12-16,18033285352731.29,2015,2014,16
2507,2014-12-15,18026487992142.56,2015,2014,15
2508,2014-12-12,17998480987314.10,2015,2014,12
2509,2014-12-11,17999196518791.71,2015,2014,11


In [ ]:
output.to_csv('shumayl_scrape.csv', index=False)
print('Saved')